In [12]:
import polars as pl
import sys
import os
import json

sys.path.append(os.path.dirname(os.path.abspath("")))

from libraries.client_stashapp import get_stashapp_client, StashAppClient
from libraries.stash_check_builder import (
    check, query, tags, performer_tags, studios, fix
)

from libraries import browser

stash = get_stashapp_client()
stash_client = StashAppClient()

base_url = "https://stash.chiefsclub.com"

dUsing stash (v0.28.1-89-g642b0f22) endpoint at http://localhost:6969/graphql
dUsing stash (v0.28.1-89-g642b0f22) endpoint at http://localhost:6969/graphql


In [13]:
# Browser functionality for injecting fix data
def check_browser_connection():
    """Check if Chrome is running with remote debugging enabled"""
    try:
        import requests
        response = requests.get('http://localhost:9222/json', timeout=5)
        if response.status_code == 200:
            pages = json.loads(response.text)
            print(f"✅ Chrome is running with remote debugging")
            print(f"📊 Found {len(pages)} open tabs/pages")
            return True
        else:
            print(f"❌ Chrome debugging port responded with status {response.status_code}")
            return False
    except Exception as e:
        print("❌ Cannot connect to Chrome debugging port")
        print("🔧 Please start Chrome with: --remote-debugging-port=9222")
        return False

def create_scenes_query_url(url_query):
    """Create a Stash query URL for scenes with Facial tag but excluding Cumshot tag"""
    import urllib.parse

    url = f"{base_url}/scenes?sortby=path&perPage=60"

    for component in url_query:
        # Convert to JSON string and use the specific URL encoding format that Stash expects
        query_json = json.dumps(component)
        # Replace curly braces with parentheses and square brackets with %5B %5D
        query_str = query_json.replace("{", "(").replace("}", ")").replace("[", "%5B").replace("]", "%5D")
        # URL encode the query string
        query_str = "&c=" + urllib.parse.quote(query_str, safe="():%5B%5D")
        url += query_str
    
    return url

def open_scenes_with_fix_data(check, check_matching_scenes):
    """Open the problematic scenes in browser with fix data injected"""
    
    if not check_matching_scenes:
        print("⚠️  No scenes found that need fixing")
        return False
    
    # Create URL for the scenes
    url = create_scenes_query_url(check["url_query"])
    
    print(f"🚀 Opening browser with {len(check_matching_scenes)} scenes that need to be fixed...")
    print(f"📋 Fix data will be injected as window.stashFixesData")
    print(f"🔗 URL: {url}")
    
    # Open the browser tab with fix data
    results = browser.open_or_update_tabs_with_data([url], check["fixes"], "stashFixesData")
    
    success = results.get(url, False)
    if success:
        print("✅ Successfully opened browser tab with fix data!")
    else:
        print("❌ Failed to open browser tab")
    
    return success

In [14]:
checks = []

In [15]:
name = "Facial without Cumshot"
checks.append(
    check()
    .name(name)
    .query(
        query().tags(
            tags()
            .include("Facial")
            .exclude("Cumshot", "Fake Cum")
            .modifier("INCLUDES_ALL")
        )
    )
    .fix(
        fix("Missing Cumshot Fix")
        .add_tags("Cumshot")
    )
    .build(stash)
)

print(f"✅ Created check: {name}")

✅ Created check: Facial without Cumshot


In [16]:
name = "Both Straight and Lesbian without Multiple Sex Scenes In A Scene"
checks.append(
    check()
    .name(name)
    .query(
        query().tags(
            tags()
            .include("Twosome (Straight)", "Twosome (Lesbian)")
            .exclude("Multiple Sex Scenes In A Scene")
            .modifier("INCLUDES_ALL")
            .depth(0)
        )
    )
    .fix(
        fix("Remove Twosome (Straight)")
        .remove_tags("Twosome (Straight)")
    )
    .fix(
        fix("Remove Twosome (Lesbian)")
        .remove_tags("Twosome (Lesbian)")
    )
    .build(stash)
)

print(f"✅ Created check: {name}")

✅ Created check: Both Straight and Lesbian without Multiple Sex Scenes In A Scene


In [17]:
name = "Both BBG and BGG without Multiple Sex Scenes In A Scene"
checks.append(
    check()
    .name(name)
    .query(
        query().tags(
            tags()
            .include("Threesome (BBG)", "Threesome (BGG)")
            .exclude("Multiple Sex Scenes In A Scene")
            .modifier("INCLUDES_ALL")
            .depth(0)
        )
    )
    .fix(
        fix("Remove Threesome (BBG)")
        .remove_tags("Threesome (BBG)")
    )
    .fix(
        fix("Remove Threesome (BGG)")
        .remove_tags("Threesome (BGG)")
    )
    .build(stash)
)

print(f"✅ Created check: {name}")

✅ Created check: Both BBG and BGG without Multiple Sex Scenes In A Scene


In [18]:
name = "Data Quality: 2 Performers No Group Makeup, Tags Indicate Straight"
checks.append(
    check()
    .name(name)
    .query(
        query()
        .performer_count("EQUALS", 2)
        .tags(
            tags()
            .modifier("INCLUDES")
            .include("Blowjob", "Facial", "Cumshot")
            .exclude("Behind the Scenes", "Full Movie", "Category: Group Makeup")
            .depth(-1)
        )
    )
    .fix(
        fix("Add Group Makeup Verified, Twosome, Twosome (Straight)")
        .add_tags("Group Makeup Verified", "Twosome", "Twosome (Straight)")
    )
    .build(stash)
)

print(f"✅ Created check: {name}")

✅ Created check: Data Quality: 2 Performers No Group Makeup, Tags Indicate Straight


In [19]:
name = "Data Quality: Straight scene with Transgender Female performer"
checks.append(
    check()
    .name(name)
    .query(
        query()
        .performer_tags(
            performer_tags()
            .include("Transgender Female")
            .modifier("INCLUDES")
            .depth(0)
        )
        .tags(
            tags()
            .include("Straight", "Twosome (Straight)")
            .exclude("Multiple Sex Scenes In A Scene")
            .modifier("INCLUDES_ALL")
            .depth(0)
        )
    )
    .fix(
        fix("Remove Straight, Twosome (Straight)")
        .remove_tags("Straight", "Twosome (Straight)")
    )
    .build(stash)
)

print(f"✅ Created check: {name}")

✅ Created check: Data Quality: Straight scene with Transgender Female performer


In [20]:
name = "Data Quality: Futanari XXX without Futanari tag"
checks.append(
    check()
    .name(name)
    .query(
        query()
        .tags(
            tags()
            .exclude("Futanari")
            .modifier("INCLUDES")
            .depth(0)
        )
        .studios(
            studios()
            .include("Futanari XXX")
            .modifier("INCLUDES")
            .depth(0)
        )
    )
    .fix(
        fix("Add Futanari tag")
        .add_tags("Futanari")
    )
    .build(stash)
)

print(f"✅ Created check: {name}")

dmatched "Futanari\ XXX" to "Futanari XXX" (46) using primary name
dmatched "Futanari\ XXX" to "Futanari XXX" (46) using primary name


✅ Created check: Data Quality: Futanari XXX without Futanari tag


In [21]:
checks_with_matching_scenes = []

for check in checks:
    matching_scenes = stash.find_scenes(check["query"], fragment=check["fragment"])
    print(f"Found {len(matching_scenes)} scenes for {check['name']}")
    if len(matching_scenes) > 0:
        checks_with_matching_scenes.append(check)

Found 0 scenes for Facial without Cumshot
Found 0 scenes for Both Straight and Lesbian without Multiple Sex Scenes In A Scene
Found 0 scenes for Both BBG and BGG without Multiple Sex Scenes In A Scene
Found 14 scenes for Data Quality: 2 Performers No Group Makeup, Tags Indicate Straight
Found 0 scenes for Data Quality: Straight scene with Transgender Female performer
Found 0 scenes for Data Quality: Futanari XXX without Futanari tag


In [22]:
# Check browser connection
browser_ready = check_browser_connection()

current_check = checks[3]
current_matching_scenes = stash.find_scenes(current_check["query"], fragment=current_check["fragment"])

if not browser_ready:
    print("⚠️  Browser not ready. Start Chrome with:")
    print('Start-Process "chrome.exe" -ArgumentList "--remote-debugging-port=9222", "--user-data-dir=C:\\\\temp\\\\chrome-debug"')
else:
    open_scenes_with_fix_data(current_check, current_matching_scenes)

✅ Chrome is running with remote debugging
📊 Found 7 open tabs/pages
🚀 Opening browser with 14 scenes that need to be fixed...
📋 Fix data will be injected as window.stashFixesData
🔗 URL: https://stash.chiefsclub.com/scenes?sortby=path&perPage=60&c=(%22type%22:%20%22performer_count%22%2C%20%22modifier%22:%20%22EQUALS%22%2C%20%22value%22:%202)&c=(%22type%22:%20%22tags%22%2C%20%22modifier%22:%20%22INCLUDES%22%2C%20%22value%22:%20(%22items%22:%20%5B(%22id%22:%20%225318%22%2C%20%22label%22:%20%22Blowjob%22)%2C%20(%22id%22:%20%225814%22%2C%20%22label%22:%20%22Facial%22)%2C%20(%22id%22:%20%225626%22%2C%20%22label%22:%20%22Cumshot%22)%5D%2C%20%22excluded%22:%20%5B(%22id%22:%20%225258%22%2C%20%22label%22:%20%22Behind%20the%20Scenes%22)%2C%20(%22id%22:%20%227635%22%2C%20%22label%22:%20%22Full%20Movie%22)%2C%20(%22id%22:%20%227777%22%2C%20%22label%22:%20%22Category:%20Group%20Makeup%22)%5D%2C%20%22depth%22:%20-1))
Got response from Chrome: 200
Found 7 existing pages
Creating new tab for https://st